# Ejercicio 2

## Exploratory Data Analysis (EDA)

In [3]:
import pandas as pd
df = pd.read_csv('diabetes.csv')

Imprimimos ahora las primeras 5 filas del data frame

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


El tamaño del data frame con el que trabajaremos será de:

In [5]:
print(df.shape)

(768, 9)


hacemos un .info para ver los datatypes y si hay celdas vacías

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


Con la siguiente linea checkeamos efectivamente que no hay datos vacios para ninguna categoría

In [22]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Guardamos los dataframes de las personas con diabetes y de las personas sanas

In [29]:
D = df[(df['Outcome'] != 0)]
H = df[(df['Outcome'] == 0)]

Mostramos en un gráfico cuantos casos del data frame tienen diabetes y cuantos no:

In [35]:
healthy_people_num = (df['Outcome'] == 0).sum()
sick_people_num = (df['Outcome'] != 0).sum()
print(healthy_people_num)
print(sick_people_num)

500
268


Poner gráfico para ver la diferencia


Vemos que el grupo de casos versus los no casos de diabetes no esta balanceado.


Luego podemos ver que hay muchos datos que tienen valor cero, y esto sabemos que no puede ser cierto, ya que por ejemplo una persona no puede tener presion arterial igual a cero.

In [40]:
import numpy as np

In [41]:
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

Ahora vemos cuantos casilleros son nulos para cada categoria

In [42]:
df.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

Poner grafico aqui si tenemos ganas



Vemos que muchos datos son nulos, esto no es de mucha ayuda, si tuvieramos un set de datos grande podríamos prescindir de la fila, pero como nuestro set no es tan grande, veremos mas adelante como tratar esto datos.

Para obtener mas datos sobre el dataframe podemos utilizar el metodo describe

In [48]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [57]:
H.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,500.000000,500.0000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.0
mean,3.298000,109.9800,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000,0.0
std,3.017185,26.1412,18.063075,14.889947,98.865289,7.689855,0.299085,11.667655,0.0
min,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.0
25%,1.000000,93.0000,62.000000,0.000000,0.000000,25.400000,0.229750,23.000000,0.0
50%,2.000000,107.0000,70.000000,21.000000,39.000000,30.050000,0.336000,27.000000,0.0
75%,5.000000,125.0000,78.000000,31.000000,105.000000,35.300000,0.561750,37.000000,0.0
max,13.000000,197.0000,122.000000,60.000000,744.000000,57.300000,2.329000,81.000000,0.0


In [58]:
D.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.0
mean,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164,1.0
std,3.741239,31.939622,21.491812,17.679711,138.689125,7.262967,0.372354,10.968254,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.088000,21.000000,1.0
25%,1.750000,119.000000,66.000000,0.000000,0.000000,30.800000,0.262500,28.000000,1.0
50%,4.000000,140.000000,74.000000,27.000000,0.000000,34.250000,0.449000,36.000000,1.0
75%,8.000000,167.000000,82.000000,36.000000,167.250000,38.775000,0.728000,44.000000,1.0
max,17.000000,199.000000,114.000000,99.000000,846.000000,67.100000,2.420000,70.000000,1.0


Anotar las correlaciones que podemos ver a ojo de esas tablas


Ahora con la informacion obtenida de los describe realizados para las personas sanas y las enfermas podemos modificar los valores nulos que vimos antes que exiten en nuestro dataframe por los valores medios dentro de cada una de las distribuciones, es decir, si a la persona que tiene diabetes le falta el parámetro insulina se reemplazará este parámetro por la media que tenga el conjunto de personas con diabetes, y lo mismo para las personas sanas.

In [70]:
def median_target(var):
    """
    This function print the mean values of the target passed for each outcome
    """   
    temp = df[df[var].notnull()]
    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    return temp

### Reemplazo de datos Nulos
#### Glucose

In [71]:
median_target('Glucose')

,Outcome,Glucose
0,0,107.0
1,1,140.0


In [75]:
df.loc[(df['Outcome'] == 0 ) & (df['Insulin'].isnull()), 'Insulin'] = 107
df.loc[(df['Outcome'] == 1 ) & (df['Insulin'].isnull()), 'Insulin'] = 140

#### BloodPressure	

In [73]:
median_target('BloodPressure')

,Outcome,BloodPressure
0,0,70.0
1,1,74.5


In [76]:
df.loc[(df['Outcome'] == 0 ) & (df['BloodPressure'].isnull()), 'BloodPressure'] = 70.0
df.loc[(df['Outcome'] == 1 ) & (df['BloodPressure'].isnull()), 'BloodPressure'] = 74.5

#### SkinThickness

In [77]:
median_target('SkinThickness')

,Outcome,SkinThickness
0,0,27.0
1,1,32.0


In [78]:
df.loc[(df['Outcome'] == 0 ) & (df['SkinThickness'].isnull()), 'SkinThickness'] = 27.0
df.loc[(df['Outcome'] == 1 ) & (df['SkinThickness'].isnull()), 'SkinThickness'] = 32.0

#### Insulin

In [79]:
median_target('Insulin')

,Outcome,Insulin
0,0,107.0
1,1,140.0


In [ ]:
df.loc[(df['Outcome'] == 0 ) & (df['Insulin'].isnull()), 'Insulin'] = 107.0
df.loc[(df['Outcome'] == 1 ) & (df['Insulin'].isnull()), 'Insulin'] = 140.0

#### BMI

In [80]:
median_target('BMI')

,Outcome,BMI
0,0,30.1
1,1,34.3


In [82]:
df.loc[(df['Outcome'] == 0 ) & (df['BMI'].isnull()), 'BMI'] = 30.1
df.loc[(df['Outcome'] == 1 ) & (df['BMI'].isnull()), 'BMI'] = 34.3

#### DiabetesPedigreeFunction

In [83]:
median_target('DiabetesPedigreeFunction')

,Outcome,DiabetesPedigreeFunction
0,0,0.336
1,1,0.449


In [84]:
df.loc[(df['Outcome'] == 0 ) & (df['DiabetesPedigreeFunction'].isnull()), 'DiabetesPedigreeFunction'] = 0.336
df.loc[(df['Outcome'] == 1 ) & (df['DiabetesPedigreeFunction'].isnull()), 'DiabetesPedigreeFunction'] = 0.449

#### Age

In [85]:
median_target('Age')

,Outcome,Age
0,0,27
1,1,36


In [86]:
df.loc[(df['Outcome'] == 0 ) & (df['Age'].isnull()), 'Age'] = 27
df.loc[(df['Outcome'] == 1 ) & (df['Age'].isnull()), 'Age'] = 36

Veamos ahora la correlacion entre las variables con un grafico:


In [60]:
def correlation_plot():
    #correlation
    correlation = df.corr()
    #tick labels
    matrix_cols = correlation.columns.tolist()
    #convert to array
    corr_array  = np.array(correlation)
    trace = go.Heatmap(z = corr_array,
                       x = matrix_cols,
                       y = matrix_cols,
                       colorscale='Viridis',
                       colorbar   = dict() ,
                      )
    layout = go.Layout(dict(title = 'Correlation Matrix for variables',
                            #autosize = False,
                            #height  = 1400,
                            #width   = 1600,
                            margin  = dict(r = 0 ,l = 100,
                                           t = 0,b = 100,
                                         ),
                            yaxis   = dict(tickfont = dict(size = 9)),
                            xaxis   = dict(tickfont = dict(size = 9)),
                           )
                      )
    fig = go.Figure(data = [trace],layout = layout)
    py.iplot(fig)

In [61]:
correlation_plot()

NameError: name 'go' is not defined